# Spam Classification NLP- LightGBM Model

Please UPVOTE this notebook if you find it insightful!

Thanks in advance.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#importing libraries
import seaborn as sns
import warnings
warnings.filterwarnings(action="ignore")
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#importing dataset
df = pd.read_csv("/kaggle/input/spam-classification-for-basic-nlp/Spam Email raw text for NLP.csv")
df.head(10)

In [ ]:
#shape of dataset
df.shape

In [ ]:
#checking for null values
df.isna().sum()

In [ ]:
#info of data
df.info()

In [ ]:
df.CATEGORY.unique()

Dependent variable(CATEGORY) is already encoded, so no need to apply encoding.

In [ ]:
sns.countplot(df["CATEGORY"])

In [ ]:
df["CATEGORY"].value_counts()

# Cleaning Text

In [ ]:
def standardize_text(df, content_field):
    df[content_field] = df[content_field].str.replace(r"http\S+", "")
    df[content_field] = df[content_field].str.replace(r"http", "")
    df[content_field] = df[content_field].str.replace(r"@\S+", "")
    df[content_field] = df[content_field].str.replace(r"[^A-Za-z0-9(),!?@\'\`\"\_\n]", " ")
    df[content_field] = df[content_field].str.replace(r"@", "at")
    df[content_field] = df[content_field].str.lower()
    return df

In [ ]:
standardize_text(df,"MESSAGE")

In [ ]:
import re
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()
corpus = []

for i in range(0, len(df)):
    message = re.sub('[^a-zA-Z]', ' ', df['MESSAGE'][i])
    message = message.split()
    message =[word for word in message if not word in set(stopwords.words('english'))]
    message = [lemmatizer.lemmatize(word) for word in message]
    message = ' '.join(message)
    corpus.append(message)

## Applying tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 3))
X = tfidf.fit_transform(corpus)
y = df["CATEGORY"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Training LightGBM Classifier

In [ ]:
# build the lightgbm model
import lightgbm as lgb
classifier = lgb.LGBMClassifier()
classifier.fit(X_train, y_train)

## Accuracy and Confusion Matrix of Model

In [ ]:
y_pred = classifier.predict(X_test)
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
acc = accuracy_score(y_pred, y_test)
report = classification_report(y_test, y_pred)
print(report)
cm = confusion_matrix(y_pred, y_test)
sns.heatmap(cm, annot=True)
print("Accuracy of LightGBM Model:", acc*100,"%")

Author: Purvit Vashishtha